In [ ]:
from ot_metasurfaces import *
%matplotlib inline

def make_grid(a,b,n):
    N = n*n
    x,y = np.meshgrid(np.linspace(a,b,n),
                      np.linspace(a,b,n))
    Y = np.zeros((N,2))
    Y[:,0] = x.flatten()
    Y[:,1] = y.flatten()
    return N,Y

def make_gaussian(n):
    N,Y = make_grid(-1,1,n)
    Y += 0.001*np.random.rand(N,2)
    nu = np.exp(-2*(Y[:,0]**2+Y[:,1]**2))
    nu *= 4/np.sum(nu)
    return N,Y,nu

def H_DH(psi):
    lag = make_laguerre(Y, psi, beta, eps=eps)
    return compute_H_DH_from_laguerre(Y,beta,lag)

def savetight(fig,fname):
    ax = fig.gca()
    ax.set_axis_off()
    plt.savefig(fname, bbox_inches='tight', pad_inches=0)

In [ ]:
# first experiment: changing the parameter beta for a discretization of size N=25
N,Y = make_grid(0,1,5)
Y += 0.001*np.random.rand(N,2)
eps = 0.01    

# initialization is independent of beta
fig = plt.figure(figsize=(20,20))
plot_laguerre(make_laguerre(Y,np.zeros(N),beta=1), linewidth=2)
savetight(fig, 'laguerre-init-N={}.png'.format(N))
        
for beta in [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 10]:
    psi,_ = solve_ot(H_DH, np.zeros(N), stoperr=1e-3, maxiter=15)
    fig = plt.figure(figsize=(20,20))
    plot_laguerre(make_laguerre(Y,psi,beta))
    savetight(fig, 'laguerre-final-N={}-beta={}.png'.format(N,beta))


In [ ]:
# second experiment: evolution of the number of Newton iterations for N=25,100, ..., 10000
beta = 0.1
eps = 0.001

plt.figure()
for n in [5,10,20,30,50,100]:
    N,Y,nu = make_gaussian(n)
    psi,err = solve_ot(H_DH, np.zeros(N), nu=nu, stoperr=1e-8, maxiter=15)
    np.savetxt('gaussian-errors-N={}.txt'.format(N), err, delimiter='\n')
    plt.semilogy(err, label='N=%d'%N)
plt.legend()
savetight(fig, 'gaussian-laguerre-convergence.png')

# save last example
fig = plt.figure(figsize=(20,20))
plot_laguerre(make_laguerre(Y,np.zeros(N),beta))
savetight(fig, 'gaussian-laguerre-initial-N={}-beta={}.png'.format(N,beta))
fig = plt.figure(figsize=(20,20))
plot_laguerre(make_laguerre(Y,psi,beta))
savetight(fig, 'gaussian-laguerre-final-N={}-beta={}.png'.format(N,beta))

In [ ]:
# Oliker-Prussner
# N = 10
# X = np.random.rand(N,3)
# w = np.zeros(N)
# w[0] = 5
# delta = 0.35
# nu = 4*np.ones(N)/N
# 
# def H(w):
#    return areas_of_power_diagram_quadric_intersection(X,w,quad)
# def Hi(w,i):
#    cells = pypower.power_diagram(X,w)
#    return area_of_cell(intersect_cell_with_quadric(X,cells[i],quad,20))
# def e(i):
#    x = np.zeros(N)
#    x[i] = 1
#    return x
#    
# while delta>0.01:
#    print("delta={}".format(delta))
#    good = False
#    while not good:
#        good = True
#        t = np.zeros(N)
#        for i in range(1,N):
#            if Hi(w,i) >= nu[i] - delta:
#                continue
#            f = lambda s: Hi(w+s*e(i),i) - nu[i]
#            # cell i does not have enough mass: we need to increase w
#            t[i] = opt.root_scalar(f, bracket=[0,10]).root
#            good = False
#        print(np.linalg.norm(H(w) - 4/N))
#        w = w + t
#    delta /= 2